# AgentCore Runtime での大容量マルチモーダルペイロードの処理

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime が Excel ファイルや画像などのマルチモーダルコンテンツを含む、最大 100MB の大容量ペイロードをどのように処理するかを実証します。AgentCore Runtime は、リッチメディアコンテンツと大きなデータセットをシームレスに処理するように設計されています。

### チュートリアル詳細

|項目| 詳細|
|:--------------------|-|:---------------------------------------------------------------------------|
| チュートリアル タイプ | 大容量ペイロードとマルチモーダル処理|
| エージェント タイプ  | シングル         |
| エージェントフレームワーク | Strands Agents |
| LLM モデル          | Anthropic Claude Sonnet 3.7 |
| チュートリアル コンポーネント | 大きなファイル処理、画像分析、Excel データ処理 |
| チュートリアル領域   | データ分析とマルチモーダル AI                                               |
| サンプルの複雑さ      | 中級                                                                     |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK|

### 主な機能

* **大容量ペイロードサポート**: 最大 100MB のサイズのファイルを処理
* **マルチモーダル処理**: Excel ファイル、画像、テキストを同時に処理
* **データ分析**: 構造化データと視覚コンテンツからインサイトを抽出
* **Base64 エンコーディング**: JSON ペイロードを通じたバイナリデータの安全な送信

## 前提条件

* Python 3.10+
* 設定済みの AWS 認証情報
* 実行中の Docker
* テスト用のサンプル Excel ファイルと画像

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## サンプルデータファイルの作成

大容量ペイロード処理を実証するために、サンプルの Excel と画像ファイルを作成しましょう：

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

# Create a large Excel file with sample sales data
np.random.seed(42)
data = {
    'Date': pd.date_range('2023-01-01', periods=1000, freq='h'),
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y'], 1000),
    'Sales': np.random.randint(1, 1000, 1000),
    'Revenue': np.random.uniform(10.0, 5000.0, 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
    'Customer_ID': np.random.randint(1000, 9999, 1000)
}

df = pd.DataFrame(data)
df.to_excel('large_sales_data.xlsx', index=False)

# Create a sample chart image
img = Image.new('RGB', (600, 500), color='white')
draw = ImageDraw.Draw(img)

# Draw a simple bar chart
products = ['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y']
values = [250, 180, 320, 150, 280]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

max_value = max(values)
bar_width = 120
start_x = 100

for i, (product, value, color) in enumerate(zip(products, values, colors)):
    x = start_x + i * (bar_width + 20)
    height = int((value / max_value) * 400)
    y = 500 - height
    
    # Draw bar
    draw.rectangle([x, y, x + bar_width, 500], fill=color)
    
    # Add labels (simplified without font)
    draw.text((x + 10, 510), product[:8], fill='black')
    draw.text((x + 10, y - 20), str(value), fill='black')

draw.text((300, 50), 'Sales Performance by Product', fill='black')
img.save('sales_chart.png')

# Check file sizes
excel_size = os.path.getsize('large_sales_data.xlsx') / (1024 * 1024)  # MB
image_size = os.path.getsize('sales_chart.png') / (1024 * 1024)  # MB

print(f"Excel file size: {excel_size:.2f} MB")
print(f"Image file size: {image_size:.2f} MB")
print(f"Total payload size: {excel_size + image_size:.2f} MB")

## マルチモーダルエージェントの作成

大容量ペイロードから Excel ファイルと画像の両方を処理できるエージェントを作成しましょう：

In [ ]:
%%writefile multimodal_data_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import base64
import io
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Initialize the model and agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
    max_tokens=16000
)

agent = Agent(
    model=model,
    system_prompt="""
    You are a data analysis assistant that can process large Excel files and images.
    When given multi-modal data, analyze both the structured data and visual content,
    then provide comprehensive insights combining both data sources.
    """
)

@app.entrypoint
def multimodal_data_processor(payload, context):
    """
    Process large multi-modal payloads containing Excel data and images.
    
    Args:
        payload: Contains prompt, excel_data (base64), image_data (base64)
        context: Runtime context information
    
    Returns:
        str: Analysis results from both data sources
    """
    prompt = payload.get("prompt", "Analyze the provided data.")
    excel_data = payload.get("excel_data", "")
    image_data = payload.get("image_data", "")
    
    print(f"=== Large Payload Processing ===")
    print(f"Session ID: {context.session_id}")
    
    if excel_data:
        print(f"Excel data size: {len(excel_data) / 1024 / 1024:.2f} MB")
    if image_data:
        print(f"Image data size: {len(image_data) / 1024 / 1024:.2f} MB")
    print(f"Excel data {excel_data}")
    print(f"Image data {image_data}")
    print(f"=== Processing Started ===")
    # Decode base64 to bytes
    excel_bytes = base64.b64decode(excel_data)
    # Decode base64 to bytes
    image_bytes = base64.b64decode(image_data)
    
    # Enhanced prompt with data context
    enhanced_prompt = f"""{prompt}
    Please analyze both data sources and provide insights.
    """
    
    response = agent(
        [{
            "document": {
                "format": "xlsx",
                "name": "excel_data",
                "source": {
                    "bytes": excel_bytes
                }
            }
        },
        {
            "image": {
                "format": "png",
                "source": {
                    "bytes": image_bytes
                }
            }
        },
        {
            "text": enhanced_prompt
        }]
    )
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## インフラストラクチャのセットアップとエージェントのデプロイ

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="multimodal_data_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="multimodal_data_agent"
)

launch_result = agentcore_runtime.launch()

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Deployment status: {status}")

print(f"Final status: {status}")

## 大容量マルチモーダルペイロードのテスト

次に、Excel データと画像の両方を含む大容量ペイロードでエージェントをテストしましょう：

In [ ]:
import base64
import uuid
import json
from IPython.display import Markdown, display

# Encode files to base64
with open('large_sales_data.xlsx', 'rb') as f:
    excel_base64 = base64.b64encode(f.read()).decode('utf-8')

with open('sales_chart.png', 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

# Create large payload
large_payload = {
    "prompt": "Analyze the sales data from the Excel file and correlate it with the chart image. Provide insights on sales performance and trends.",
    "excel_data": excel_base64,
    "image_data": image_base64
}

session_id = str(uuid.uuid4())
print(f"📊 Processing large multi-modal payload...")
print(f"📋 Session ID: {session_id}")
print(f"📄 Excel size: {len(excel_base64) / 1024 / 1024:.2f} MB")
print(f"🖼️ Image size: {len(image_base64) / 1024 / 1024:.2f} MB")
print(f"📦 Total payload: {len(json.dumps(large_payload)) / 1024 / 1024:.2f} MB\n")

# Invoke agent with large payload
invoke_response = agentcore_runtime.invoke(
    large_payload,
    session_id=session_id
)
final_response = ""
for r in invoke_response['response']:
    final_response += r.decode("utf-8")
response_data = json.loads(final_response)
display(Markdown(response_data))

## リソースのクリーンアップ

In [ ]:
import boto3

# Clean up AWS resources
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

# Delete AgentCore Runtime
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# Delete ECR repository
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Clean up local files
os.remove('large_sales_data.xlsx')
os.remove('sales_chart.png')

print("✅ Cleanup completed!")

# おめでとうございます！

Amazon Bedrock AgentCore Runtime を使用した大容量マルチモーダルペイロードの処理に成功しました！

## 学んだこと：

### 大容量ペイロード処理
* **100MB サポート**: AgentCore Runtime は最大 100MB のペイロードを処理可能
* **Base64 エンコーディング**: JSON ペイロードを通じたバイナリデータの安全な送信
* **効率的な処理**: 大きなデータ処理に最適化されたランタイム

### マルチモーダル機能
* **Excel 分析**: スプレッドシートからの構造化データの処理
* **画像処理**: 視覚的コンテンツとチャートの分析
* **統合分析**: 複数のデータタイプからのインサイトの相関

### 主な利点
* **豊富なデータ処理**: 複雑で複数形式のデータセットを処理
* **スケーラブルなアーキテクチャ**: 大きなワークロードに対応したランタイム設計
* **ツール統合**: 特化したデータ処理用のカスタムツール
* **エンタープライズ対応**: 機密ビジネスデータの安全な処理

これは、複雑なビジネスインテリジェンスとデータ分析アプリケーションに理想的な、複数のデータモダリティを持つエンタープライズスケールのデータ処理タスクを処理する AgentCore Runtime の機能を実証しています。